In [ ]:
import firebase_admin
from firebase_admin import credentials, storage
from firebase_admin import db
import pandas as pd
import os
import dotenv
dotenv.load_dotenv()

## Firebase Credentials

In [ ]:
service_account_info ={
    "type": os.getenv('FIREBASE_TYPE'),
    "project_id": os.getenv('FIREBASE_PROJECT_ID'),
    "private_key_id": os.getenv('FIREBASE_PRIVATE_KEY_ID'),
    "private_key": os.getenv('FIREBASE_PRIVATE_KEY'),
    "client_email": os.getenv('FIREBASE_CLIENT_EMAIL'),
    "client_id": os.getenv('FIREBASE_CLIENT_ID'),
    "auth_uri": os.getenv('FIREBASE_AUTH_URI'),
    "token_uri": os.getenv('FIREBASE_TOKEN_URI'),
    "auth_provider_x509_cert_url": os.getenv('FIREBASE_AUTH_PROVIDER_X509_CERT_URL'),
    "client_x509_cert_url": os.getenv('FIREBASE_CLIENT_X509_CERT_URL'),
    "universe_domain": os.getenv('FIREBASE_UNIVERSE_DOMAIN')
  }

In [ ]:
cred = credentials.Certificate(service_account_info)
firebase_admin.initialize_app(cred,{
    'storageBucket': 'coffee-shop-app-a4152.appspot.com',
     'databaseURL': 'https://coffee-shop-app-a4152-default-rtdb.firebaseio.com/'
})

In [ ]:
bucket = storage.bucket()


In [ ]:
image_folder_path = './products/images/'


In [ ]:
products_collection = db.reference('products')


In [ ]:
df = pd.read_json('products/products.jsonl',lines=True)
df.head(2)


In [ ]:
def upload_image(bucket, image_path):
    image_name = image_path.split('/')[-1]
    blob = bucket.blob(f'product_images/{image_name}')
    # Upload image
    blob.upload_from_filename(image_path)
    # Make the image publicly accessible and get its URL
    blob.make_public()
    return blob.public_url

In [ ]:
for index, row in df.iterrows():
    print(index, row['name'])
    
    image_path = os.path.join(image_folder_path,row['image_path'])
    
    image_url = upload_image(bucket,image_path)
    product_data = row.to_dict()
    product_data.pop('image_path')
    product_data['image_url']= image_url
    
    # Add to Firestore
    products_collection.push().set(product_data)